In [1]:
!pip install tkinter
!pip install email-validator


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


Defaulting to user installation because normal site-packages is not writeable

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



  Obtaining dependency information for email-validator from https://files.pythonhosted.org/packages/d7/ee/bf0adb559ad3c786f12bcbc9296b3f5675f529199bef03e2df281fa1fadb/email_validator-2.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for dnspython>=2.0.0 from https://files.pythonhosted.org/packages/87/a1/8c5287991ddb8d3e4662f71356d9656d91ab3a36618c3dd11b280df0d255/dnspython-2.6.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   ------- -------------------------------- 61.4/307.7 kB 3.2 MB/s eta 0:00:01
   -------------- ------------------------- 112.6/307.7 kB 1.6 MB/s eta 0:00:01
   -------------------------- ------------- 204.8/307.7 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------  307.2/307.7 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 307.7/307.7 kB 1.6 MB/s eta 0:00:00


In [ ]:
import tkinter as tk
from tkinter import messagebox
import smtplib
import ssl
import sqlite3
from email.message import EmailMessage

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('email_app.db')
cursor = conn.cursor()

# Create a table for user settings if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS user_settings (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    email TEXT,
    smtp_server TEXT,
    smtp_port INTEGER,
    password TEXT
)
''')

# Create a table to store sent emails
cursor.execute('''
CREATE TABLE IF NOT EXISTS sent_emails (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    recipient TEXT,
    subject TEXT,
    body TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Commit changes to the database
conn.commit()

# Function to store SMTP settings in the database
def save_settings(email, smtp_server, smtp_port, password):
    cursor.execute('DELETE FROM user_settings')  # Clear previous settings
    cursor.execute('INSERT INTO user_settings (email, smtp_server, smtp_port, password) VALUES (?, ?, ?, ?)',
                   (email, smtp_server, smtp_port, password))
    conn.commit()

# Function to retrieve SMTP settings from the database
def get_settings():
    cursor.execute('SELECT email, smtp_server, smtp_port, password FROM user_settings')
    return cursor.fetchone()

# Function to send an email
def send_email(recipient, subject, body):
    settings = get_settings()
    if settings is None:
        messagebox.showerror("Error", "Please configure your SMTP settings first.")
        return

    sender_email, smtp_server, smtp_port, password = settings

    # Create the email
    msg = EmailMessage()
    msg['From'] = sender_email
    msg['To'] = recipient
    msg['Subject'] = subject
    msg.set_content(body)

    try:
        # Connect to the SMTP server and send the email
        context = ssl.create_default_context()
        with smtplib.SMTP_SSL(smtp_server, smtp_port, context=context) as server:
            server.login(sender_email, password)
            server.send_message(msg)

        # Save the sent email to the database
        cursor.execute('INSERT INTO sent_emails (recipient, subject, body) VALUES (?, ?, ?)', 
                       (recipient, subject, body))
        conn.commit()

        messagebox.showinfo("Success", "Email sent successfully!")

    except Exception as e:
        messagebox.showerror("Error", f"Failed to send email: {e}")

# Function to configure SMTP settings
def configure_smtp():
    def save_and_close():
        email = email_entry.get()
        smtp_server = smtp_server_entry.get()
        smtp_port = smtp_port_entry.get()
        password = password_entry.get()
        
        save_settings(email, smtp_server, smtp_port, password)
        smtp_window.destroy()
        messagebox.showinfo("Success", "SMTP settings saved successfully!")

    smtp_window = tk.Toplevel(root)
    smtp_window.title("Configure SMTP Settings")

    tk.Label(smtp_window, text="Email:").pack(pady=5)
    email_entry = tk.Entry(smtp_window)
    email_entry.pack(pady=5)

    tk.Label(smtp_window, text="SMTP Server:").pack(pady=5)
    smtp_server_entry = tk.Entry(smtp_window)
    smtp_server_entry.pack(pady=5)

    tk.Label(smtp_window, text="SMTP Port:").pack(pady=5)
    smtp_port_entry = tk.Entry(smtp_window)
    smtp_port_entry.pack(pady=5)

    tk.Label(smtp_window, text="Password:").pack(pady=5)
    password_entry = tk.Entry(smtp_window, show="*")
    password_entry.pack(pady=5)

    tk.Button(smtp_window, text="Save", command=save_and_close).pack(pady=10)

# Function to show sent email records
def show_sent_emails():
    cursor.execute('SELECT recipient, subject, timestamp FROM sent_emails')
    records = cursor.fetchall()

    history_window = tk.Toplevel(root)
    history_window.title("Sent Emails")

    for record in records:
        recipient, subject, timestamp = record
        tk.Label(history_window, text=f"To: {recipient} | Subject: {subject} | Sent on: {timestamp}").pack(pady=5)

# Main application GUI
root = tk.Tk()
root.title("Mail Application")
root.geometry("500x400")

tk.Label(root, text="Recipient Email:").pack(pady=5)
recipient_entry = tk.Entry(root)
recipient_entry.pack(pady=5)

tk.Label(root, text="Subject:").pack(pady=5)
subject_entry = tk.Entry(root)
subject_entry.pack(pady=5)

tk.Label(root, text="Body:").pack(pady=5)
body_text = tk.Text(root, height=10)
body_text.pack(pady=5)

tk.Button(root, text="Send Email", command=lambda: send_email(recipient_entry.get(), subject_entry.get(), body_text.get("1.0", tk.END))).pack(pady=10)
tk.Button(root, text="Configure SMTP", command=configure_smtp).pack(pady=5)
tk.Button(root, text="View Sent Emails", command=show_sent_emails).pack(pady=5)

# Run the application
root.mainloop()

# Close the database connection when the app closes
conn.close()
